# Interactive weather dashboard in Python

<img src="data/images/partners.png" width="50%"/>





### Table of content

1. [Getting started](#section-1)
    - 1.1 Introduction to Jupyter notebook
    - 1.2 Introduction to REST APIs in Python
2. [Introduction to Application Programming Interfaces (APIs)](#section-2)
3. [TAHMO API: Weather station data](#section-3)
4. [Open-Meteo: Numerical Weather Forecast API](#section-4)
5. []

## 1. Getting started <a name="section-1"></a>

### 1.1 Jupyter Notebooks
[Jupyter Notebooks](https://jupyter-notebook.readthedocs.io/en/latest/) are interactive computing environments that allow you to create and share documents containing live code, equations, visualizations, and narrative text. They are widely used in data science, research, and education due to their versatility and ease of use.

Jupyter Notebooks consist of cells which can contain code, text, or visualizations.

#### Running a Cell:
Click on the cell below, and you'll notice a border around it. To run the code within the cell, press **Shift + Enter** or use the "Run" button in the toolbar above.


In [1]:
print('Welcome to the training!')

Welcome to the training!


#### Creating variables:

In [2]:
var = "let's start coding"

print(var)

let's start coding


#### Use functions

In [3]:
def add(a,b):
    return a+b

print(add(1,2))

3


#### Saving and Closing:
Remember to save your work by using **Ctrl + S**. You can close a notebook when you're done, and the changes will be saved.

## 2.  Introduction to Application Programming Interfaces (APIs)


### 2.1 REST APIs

An API (Application Programming Interface) is a protocol that defines how systems can communicate with each other. A REST API is built following the design principles of Representational State Transfer (REST). REST is very flexible, and therefore, it can be found all over the internet. It uses standard HTTP protocols, which are:


<img src="data/images/api.png" width="50%"/>



- **GET**: Retrieve data from a specified resource.

- **POST**: Create a new resource.

- **PUT**: Update an existing resource.

- **DELETE**: Remove a resource.

An API needs an endpoint, which is a specific URL to which the API sends requests and from which it receives responses. In simpler terms, an API endpoint is a designated route or path on a server that the API uses to perform a particular function. Each endpoint represents a specific operation or resource in the API.


In [4]:
import requests

url = 'https://www.google.com'

r = requests.get(url)

print(r.status_code)


200


When a request fails or was successful, a HTTP status code is returned. Here is a list of status code that give information about the status of the request

- **200**: OK
The request was successful. 

- **400**: Bad request
The request cannot be fulfilled due to incorrect syntax or invalid parameters.

- **401**: Unauthorized 
Authentication is required, and the provided credentials are invalid.

- **403**: Forbidden
The server understood the request, but it refuses to authorize it.

- **404**: Not Found
The requested resource could not be found on the server.

- **500**: Internal Server Error
The requested resource could not be found on the server.



In [5]:
# Define a dictionary 'params' with key-value pairs to be inluded in the query sparameters
params = {"key1": "value1", "key2": "value2"}

# Define a dictionary 'headers' to include additional headers int the GET request
headers = {"user-name": "password123"}

# Use the 'request.get()' method to make a GET request to the specified URL 'https://httpbin.org/get'
# Include the defined 'params' and 'headers' in the request
r = requests.get('https://httpbin.org/get', params=params, headers=headers)

print(r.url)

https://httpbin.org/get?key1=value1&key2=value2


## 3. TAHMO station API <a name="section-2"></a>

In this section we will use the TAHMO API endpoint and retrieve data for a variety of variables. We will create an interactive visualization of the precipitation that was measured throughout this year at one of the stations. The Trans-African Hydro-Meteorological Observatory (TAHMO) maintains a network of weather stations across Africa. The data of these stations can be retrieved using the API. We can use the API-V2 client that can be found on the TAHMO GitHub page (https://github.com/TAHMO/API-V2-Python-examples). 

In [6]:

# Look around at the TAHMO website

from IPython.display import IFrame
IFrame("https://tahmo.org/", '75%',400)

In [7]:
# Import the TAHMO module
import TAHMO

# The demo credentials listed below give you access to three pre-defined stations. 
api = TAHMO.apiWrapper()

# set the credentials
api.setCredentials('demo', 'DemoPassword1!')

In the cell below we can list all the TAHMO stations that we have access to. Also we can list al the variables that are recorded by the weather stations.

In [8]:
# list other stations that are available
stations = api.getStations()
print('Account has access to stations: %s' % ', '.join(list(stations)))

API request: services/assets/v2/stations
Account has access to stations: TA00134, TA00252, TA00567


In [9]:
list(stations)

['TA00134', 'TA00252', 'TA00567']

### 3.2 Variables

In [10]:
# list available variables

variables = api.getVariables()

for variable in variables:
    print(f'{variables[variable]["description"]} {variables[variable]["units"]} with shortcode "{variables[variable]["shortcode"]}"')

API request: services/assets/v2/variables


Atmospheric pressure kPa with shortcode "ap"
Depth of water mm with shortcode "dw"
Electrical conductivity of precipitation mS/cm with shortcode "ec"
Electrical conductivity of water mS/cm with shortcode "ew"
Lightning distance km with shortcode "ld"
Lightning events - with shortcode "le"
Shortwave radiation W/m2 with shortcode "ra"
Soil moisture content m3/m3 with shortcode "sm"
Soil temperature degrees Celsius with shortcode "st"
Surface air temperature degrees Celsius with shortcode "te"
Vapor pressure kPa with shortcode "vp"
Wind gusts m/s with shortcode "wg"
Wind speed m/s with shortcode "ws"
Temperature of humidity sensor degrees Celsius with shortcode "ht"
X-axis level degrees with shortcode "tx"
Y-axis level degrees with shortcode "ty"
Logger battery percentage - with shortcode "lb"
Logger reference pressure kPa with shortcode "lp"
Logger temperature degrees Celsius with shortcode "lt"
Cumulative precipitation mm with shortcode "cp"
Water level m with shortcode "wl"
Water veloc

Let's take one of the stations and retrieve the name of the weather stations and the geographical coordinates of the station:

### 3.2	Retrieve and plot daily precipitation data

In [11]:
# choose a station
station = 'TA00567'

# get the data
station_data = api.getStations()[station]

print()
print( f"Station name =  {station_data['location']['name']}")
print( f"Longitude =  {station_data['location']['longitude']:.02f}")
print( f"Latitude =  {station_data['location']['latitude']:.02f}")

API request: services/assets/v2/stations

Station name =  Accra Girls SHS
Longitude =  -0.19
Latitude =  5.60


In [12]:
# Example 3: Retrieve a pandas dataframe containing the time serie of surface air observations and save to CSV file.

startDate = '2023-01-01'
endDate = '2023-11-22'
variables = ['pr']

df_tahmo = api.getMeasurements(station, startDate=startDate, endDate=endDate, variables=variables)
df_tahmo.index.name = 'Timestamp'

df_tahmo.head()


API request: services/measurements/v2/stations/TA00567/measurements/controlled


,pr
Timestamp,
2023-01-01 00:00:00+00:00,0.0
2023-01-01 00:05:00+00:00,0.0
2023-01-01 00:10:00+00:00,0.0
2023-01-01 00:15:00+00:00,0.0
2023-01-01 00:20:00+00:00,0.0


In [13]:
import pandas as pd

def process_tahmo_precip_data(df):
    """Load the precipitation data from the TAHMO API and return a pandas dataframe"""
    df = df.reset_index().rename(columns={"Timestamp" : "date", "pr": "precipitation"})
    df['date'] = pd.to_datetime(df['date'])
    df.loc[:,'date'] = df['date'].dt.date
    df = df.groupby('date').sum().reset_index().dropna()
    df['date'] = pd.to_datetime(df['date'])
    return df

df_tahmo = process_tahmo_precip_data(df_tahmo)
df_tahmo.head()



,date,precipitation
0,2023-01-01,0.0
1,2023-01-02,0.0
2,2023-01-03,0.0
3,2023-01-04,0.0
4,2023-01-05,0.0


[Vega-Altair](https://altair-viz.github.io/)  is a handy Python library that allows you to create statistical visualizations using declarative principles without complex programming code. It provides a straightforward way to quickly generate various types of charts.

Altair is built on  [Vega-Lite](https://vega.github.io/vega-lite/), which is a Grammar of Interactive Graphics. Vega-Altair provides a user-friendly way to use this through Python, storing the graphical specifications in JSON (JavaScript Object Notation) format. You can directly view these specifications in any web browser, and coding is easy in JupyterLab, Jupyter Notebook, Microsoft VS-Code, and Google Colab.

In [14]:
import altair as alt

timeseries_tahmo =  alt.Chart(df_tahmo).mark_bar().encode(x="date", y="precipitation", tooltip=['precipitation', 'date']).properties(width=1200, height=200).interactive()
timeseries_tahmo

alt.Chart(...)

### Exercise:

Try to retrieve data of another variable, the temperature measured at one of the TAHMO stations and visualize this using a Chart with a line mark:
```python
alt.Chart(df).mark_line()
```

In [15]:

def load_temp_data():
    
    # write code here

    return 


# make an api call to get the data


# plot the data



## 4. Open-Meteo weather API

[Open-Meteo](https://github.com/open-meteo/open-meteo) is an open-source weather API and offers free access for non-commercial use. It includes hourly forecasts up to 16 days, but also historic weather. See the website below for more details:


In [16]:
from IPython.display import IFrame
IFrame("https://open-meteo.com/", "75%", 400)

In [17]:
import requests


def get_ecmwf_precipitation_forecast(lon, lat):
    """Retrieve the ECMWF precipitation forecast from the Open-Meteo API and return a JSON object"""

    base_url = "https://api.open-meteo.com/v1/forecast"
    
    # Specify the parameters for the ECMWF precipitation forecast
    params = {
        "longitude" : lon,
        "latitude" : lat,
        "daily" : "precipitation_sum",
        "past_days" : 90,
        "timezone" : "auto",
        "hourly" : "precipitation",
        "start" : "current",
        "forecast_days" : 15,
        "models" : "ecmwf_ifs04"}

    try:
        # Make a request to the Open-Meteo API
        response = requests.get(base_url, params=params)
        data = response.json()
        return data
    except requests.RequestException as e:
        print(f"Error: {e}")

data_ecmwf = get_ecmwf_precipitation_forecast(lon=station_data['location']['longitude'], lat=station_data['location']['latitude'])



In [18]:
def process_ecmwf_precip_data(data):
    """Load the precipitation data from the Open-Meteo API and return a pandas dataframe"""
    df = pd.DataFrame.from_dict(data['hourly'])
    df['time'] = pd.to_datetime(df['time'])
    df.loc[:,'date'] = df['time'].dt.date
    df['date'] = pd.to_datetime(df['date'])
    df = df[['date', 'precipitation']].dropna()
    df = df.groupby('date').sum().reset_index()# .set_index('date')
    return df

df_ecmwf = process_ecmwf_precip_data(data_ecmwf)
df_ecmwf.head()

,date,precipitation
0,2023-08-28,3.3
1,2023-08-29,1.2
2,2023-08-30,13.2
3,2023-08-31,1.8
4,2023-09-01,1.8


In [19]:
import datetime

timeseries_ecmwf =  alt.Chart(df_ecmwf).mark_bar(color='orange').encode(x="date", y="precipitation", tooltip=['precipitation', 'date'])

rule = alt.Chart(pd.DataFrame({
  'date': [datetime.datetime.now().strftime("%Y-%m-%d")],
  'color': ['black']
})).mark_rule().encode(x='date:T') 

chart  = rule + timeseries_ecmwf 

chart.properties(width=1200, height=300).interactive()

alt.LayerChart(...)

In [20]:
import openmeteo_requests
from openmeteo_sdk.Variable import Variable
from openmeteo_sdk.Aggregation import Aggregation
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

def get_ecmwf_precipitation_ensemble(lon, lat):

	"""Retrieve the ECMWF precipitation forecast from the Open-Meteo API and return a JSON object"""
	
	url = "https://ensemble-api.open-meteo.com/v1/ensemble"
	
	params = {
		"latitude": lat,
		"longitude": lon,
		"forecast_days": 5,
		"past_days": 30,
		"hourly": "precipitation",
		"models": "ecmwf_ifs04"
	}
	responses = openmeteo.weather_api(url, params=params)

	response = responses[0]
	
	# Process hourly data
	hourly = response.Hourly()
	hourly_variables = list(map(lambda i: hourly.Variables(i), range(0, hourly.VariablesLength())))
	hourly_precipitation = filter(lambda x: x.Variable() == Variable.precipitation, hourly_variables)

	hourly_data = {"date": pd.date_range(
		start = pd.to_datetime(hourly.Time(), unit = "s"),
		end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}
	# Process all members
	for variable in hourly_precipitation:
		member = variable.EnsembleMember()
		hourly_data[f"precipitation_member{member}"] = variable.ValuesAsNumpy()

	df = pd.DataFrame(data=hourly_data)
	return df

df_hourly = get_ecmwf_precipitation_ensemble(station_data['location']['longitude'], station_data['location']['latitude'])


In [21]:
def process_ecmwf_ensemble_precip_data(df):
    """Load the precipitation data from the Open-Meteo API and return a pandas dataframe"""
    df = df.rename(columns={"date": "Timestamp"})
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    df.loc[:,'date'] = df['Timestamp'].dt.date
    df['date'] = pd.to_datetime(df['date'])
    df = df.drop(columns=['Timestamp'])
    df = df.groupby('date').sum().reset_index()
    
    return df

df_ecmwf_ensemble = process_ecmwf_ensemble_precip_data(df_hourly)


In [22]:

ensemble_df = pd.DataFrame(data={'min' : df_ecmwf_ensemble.set_index('date').min(axis=1), 'max' : df_ecmwf_ensemble.set_index('date').max(axis=1), 'mean' : df_ecmwf_ensemble.set_index('date').mean(axis=1)}).reset_index()

area = alt.Chart(ensemble_df).mark_area(opacity=0.25, color='orange').encode(x='date', y='min', y2='max').properties(width=1200, height=300).interactive()

bar = alt.Chart(ensemble_df).mark_bar(color='orange').encode(x='date', y='mean', tooltip=['mean', 'date'])


rule = alt.Chart(pd.DataFrame({
  'date': [datetime.datetime.now().strftime("%Y-%m-%d")],
  'color': ['black']
})).mark_rule().encode(x='date:T') 



chart = area + rule + bar
chart.properties(width=1200, height=300).interactive()


alt.LayerChart(...)

## 4. Introduction to web apps in Solara 

[Solara](https://solara.dev/) is a python library for data-focused web apps which you can run in a Jupyter notebook as well as in production-grade web frameworks (FastAPI, Starlette, Flask, ...). It uses IPywidgets for UI components which saves you from having to learn Javascript and CSS. 

In [23]:
station_list = ["TA00134", "TA00252", "TA00567"]

# station "TA00134" is empty, therefore we remove it from the list
station_list.remove("TA00134")

station_data = {}

for station in station_list:
    station_data[station] = api.getStations()[station]

API request: services/assets/v2/stations


API request: services/assets/v2/stations


In [24]:
import solara
import ipyleaflet
import TAHMO
from ipywidgets import HTML

# Create a TAHMO API wrapper and set credentials
api = TAHMO.apiWrapper()
api.setCredentials('demo', 'DemoPassword1!')

station_default = 'TA00252'
center_default = (station_data[station_default]['location']['latitude'], station_data[station_default]['location']['longitude'])
zoom_default = 9

# Define reactive variables for station data
station = solara.reactive(station_default)
zoom = solara.reactive(zoom_default)
center = solara.reactive(center_default)

def set_station(value):
    station.value = value
    center.value = (station_data[value]['location']['latitude'], station_data[value]['location']['longitude'])

@ solara.component
def StationSelect():
    """Solara component for a station selection dropdown."""
    solara.Select(label="station", values=station_list, value=station.value, on_value=set_station, style={"z-index": "10000"})
    
@solara.component
def View():
    """Solara component for displaying a map view with a marker for the selected station."""
    ipyleaflet.Map.element(center=center.value,
                           zoom=9,
                           on_center=center.set,
                        scroll_wheel_zoom=True, 
                        layers=[ipyleaflet.TileLayer.element(url=ipyleaflet.basemaps.OpenStreetMap.Mapnik.build_url())] + [ipyleaflet.Marker.element(location=(station_data[s]['location']['latitude'], station_data[s]['location']['longitude']), draggable=False) for s in station_list] 
                        )

@solara.component
def Page():
    """Solara component for a page with two cards: View and StationSelect."""
    with solara.Column(style={"min-width": "500px", "height": "500px"}):
        with solara.Row():
            StationSelect()
        with solara.Card():
            View()

Page()

Cannot show ipywidgets in text

In [25]:
import pandas as pd



def set_station(value):
    station.value = value
    
@ solara.component
def StationSelect():
    """Solara component for a station selection dropdown."""
    solara.Select(label="station", values=station_list, value=station.value, on_value=set_station)

def request_precip_data(station, variables=['pr'], startDate='2023-01-01', endDate='2023-11-22'):
    """Request precipitation data from the TAHMO API and return a pandas dataframe."""
    df = api.getMeasurements(station, startDate=startDate, endDate=endDate, variables=variables)
    if df.empty:
        df = pd.DataFrame(columns=['date', 'precipitation'])
        return df
    else:
        df.index.name = 'Timestamp'
        df = df.reset_index()
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df.loc[:,'date'] = df['Timestamp'].dt.date
        df = df.drop(columns=['Timestamp']).groupby('date').max().reset_index().dropna()
        df['date'] = pd.to_datetime(df['date'])
        df = df.rename(columns={"pr": "precipitation"})
        return df

@solara.component
def Timeseries():
    
    """Solara component for a timeseries chart of precipitation."""	
    variables = ['pr']
    today = datetime.datetime.now()
    startDate = today - datetime.timedelta(days=30)
    df_tahmo = api.getMeasurements(station.value, startDate=startDate.strftime("%Y-%m-%d"), endDate=today.strftime("%Y-%m-%d"), variables=variables)
    df_tahmo.index.name = 'Timestamp'
    df_tahmo = process_tahmo_precip_data(df_tahmo)
    bar_tahmo =  alt.Chart(df_tahmo).mark_bar(opacity=0.75,).encode(x="date", y="precipitation", tooltip=['precipitation', 'date']).interactive()
    df_hourly = get_ecmwf_precipitation_ensemble(station_data[station.value]['location']['longitude'], station_data[station.value]['location']['latitude'])
    df_ecmwf_ensemble = process_ecmwf_ensemble_precip_data(df_hourly)
    ensemble_df = pd.DataFrame(data={'min' : df_ecmwf_ensemble.set_index('date').min(axis=1), 'max' : df_ecmwf_ensemble.set_index('date').max(axis=1), 'mean' : df_ecmwf_ensemble.set_index('date').mean(axis=1)}).reset_index()
    area_ecmwf = alt.Chart(ensemble_df).mark_area(opacity=0.25, color='orange').encode(x='date', y='min', y2='max').interactive()
    bar_ecmwf = alt.Chart(ensemble_df).mark_bar(opacity=0.75, color='orange').encode(x='date', y='mean', tooltip=['mean', 'date'])
    rule = alt.Chart(pd.DataFrame({'date': [today.strftime("%Y-%m-%d")], 'color': ['black']})).mark_rule().encode(x='date:T') 
    chart = area_ecmwf + rule + bar_tahmo + bar_ecmwf
    solara.display(chart.properties(width=1200, height=300).interactive())

@solara.component
def Page():
    """Solara component for a page with two cards: View and StationSelect."""
    with solara.Column(style={"min-width": "500px", "height": "500px"}):
        with solara.Row():
            StationSelect()
        with solara.Card():
            Timeseries()

Page()

API request: services/measurements/v2/stations/TA00252/measurements/controlled


Cannot show ipywidgets in text

In [26]:
import solara
import ipyleaflet
import TAHMO
from ipywidgets import HTML

# Create a TAHMO API wrapper and set credentials
api = TAHMO.apiWrapper()
api.setCredentials('demo', 'DemoPassword1!')


station_default = 'TA00252'
center_default = (station_data[station_default]['location']['latitude'], station_data[station_default]['location']['longitude'])
zoom_default = 9


# Define reactive variables for station data
station = solara.reactive(station_default)
zoom = solara.reactive(zoom_default)
center = solara.reactive(center_default)


def set_station(value):
    station.value = value
    center.value = (station_data[value]['location']['latitude'], station_data[value]['location']['longitude'])

@ solara.component
def StationSelect():
    """Solara component for a station selection dropdown."""
    solara.Select(label="station", values=station_list, value=station.value, on_value=set_station, style={"z-index": "10000"})
    
@solara.component
def View():
    """Solara component for displaying a map view with a marker for the selected station."""
    
    ipyleaflet.Map.element(center=center.value,
                           zoom=9,
                           on_center=center.set,
                        scroll_wheel_zoom=True, 
                        layers=[ipyleaflet.TileLayer.element(url=ipyleaflet.basemaps.OpenStreetMap.Mapnik.build_url())] + [ipyleaflet.Marker.element(location=(station_data[s]['location']['latitude'], station_data[s]['location']['longitude']), draggable=False) for s in station_list] 
                        )

    
def request_precip_data(station, variables=['pr'], startDate='2023-01-01', endDate='2023-11-22'):
    """Request precipitation data from the TAHMO API and return a pandas dataframe."""
    df = api.getMeasurements(station, startDate=startDate, endDate=endDate, variables=variables)
    if df.empty:
        df = pd.DataFrame(columns=['date', 'precipitation'])
        return df
    else:
        df.index.name = 'Timestamp'
        df = df.reset_index()
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        df.loc[:,'date'] = df['Timestamp'].dt.date
        df = df.drop(columns=['Timestamp']).groupby('date').max().reset_index().dropna()
        df['date'] = pd.to_datetime(df['date'])
        df = df.rename(columns={"pr": "precipitation"})
        return df


@solara.component
def Timeseries():
    
    """Solara component for a timeseries chart of precipitation."""	
    variables = ['pr']
    today = datetime.datetime.now()
    startDate = today - datetime.timedelta(days=30)
    df_tahmo = api.getMeasurements(station.value, startDate=startDate.strftime("%Y-%m-%d"), endDate=today.strftime("%Y-%m-%d"), variables=variables)
    df_tahmo.index.name = 'Timestamp'
    df_tahmo = process_tahmo_precip_data(df_tahmo)
    bar_tahmo =  alt.Chart(df_tahmo).mark_bar(opacity=0.75,).encode(x="date", y="precipitation", tooltip=['precipitation', 'date']).interactive()
    df_hourly = get_ecmwf_precipitation_ensemble(station_data[station.value]['location']['longitude'], station_data[station.value]['location']['latitude'])
    df_ecmwf_ensemble = process_ecmwf_ensemble_precip_data(df_hourly)
    ensemble_df = pd.DataFrame(data={'min' : df_ecmwf_ensemble.set_index('date').min(axis=1), 'max' : df_ecmwf_ensemble.set_index('date').max(axis=1), 'mean' : df_ecmwf_ensemble.set_index('date').mean(axis=1)}).reset_index()
    area_ecmwf = alt.Chart(ensemble_df).mark_area(opacity=0.25, color='orange').encode(x='date', y='min', y2='max').interactive()
    bar_ecmwf = alt.Chart(ensemble_df).mark_bar(opacity=0.75, color='orange').encode(x='date', y='mean', tooltip=['mean', 'date'])
    rule = alt.Chart(pd.DataFrame({'date': [today.strftime("%Y-%m-%d")], 'color': ['black']})).mark_rule().encode(x='date:T') 
    chart = area_ecmwf + rule + bar_tahmo + bar_ecmwf
    solara.display(chart.properties(width=1200, height=300).interactive())

@solara.component
def Page():
    """Solara component for a page with two cards: View and StationSelect."""
    with solara.Column(style={"min-width": "500px", "height": "500px"}):
        with solara.Row():
            StationSelect()
        with solara.Columns([1, 2]):
            with solara.Card():
                View()
            with solara.Card():
                Timeseries()

Page()

API request: services/measurements/v2/stations/TA00252/measurements/controlled


Cannot show ipywidgets in text

API request: services/measurements/v2/stations/TA00567/measurements/controlled
